In [2]:
pip install yfinance

     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ----------------- ---------------------- 1.3/3.0 MB 7.4 MB/s eta 0:00:01
     ---------------------------------- ----- 2.6/3.0 MB 7.5 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 6.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.9-py3-none-any.whl size=139095 sha256=6759e974d65b3c1aeaa542f885875122363446d4b3d2bdbf75b0cfb3a1ff0da0
  Stored in directory: c:\users\np.mohsenpak\appdata\local\pip\cache\wheels\43\ef\2d\2c51d496bf084945ffdf838b4cc8767b8ba1cc20eb41588831
Successfully built peewee
Note: you may need to restart the kernel to use upd

In [6]:
import yfinance as yf
data = yf.download("AAPL", start="2023-01-01", end="2024-01-01")
print(data.columns)
print(data.head())


[*********************100%***********************]  1 of 1 completed

MultiIndex([( 'Close', 'AAPL'),
            (  'High', 'AAPL'),
            (   'Low', 'AAPL'),
            (  'Open', 'AAPL'),
            ('Volume', 'AAPL')],
           names=['Price', 'Ticker'])
Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2023-01-03  123.632530  129.395518  122.742873  128.782649  112117500
2023-01-04  124.907715  127.181283  123.642427  125.431622   89113600
2023-01-05  123.583107  126.301500  123.326101  125.668857   80962700
2023-01-06  128.130203  128.792501  123.454572  124.561702   87754700
2023-01-09  128.654144  131.876686  128.397138  128.970474   70790800


In [2]:
import yfinance as yf
import sqlite3

# Step 1: Download the data
data = yf.download("AAPL", start="2023-01-01", end="2024-01-01")

# Step 2: Flatten the MultiIndex by resetting the columns
data.columns = [f'{col[0]}_{col[1]}' for col in data.columns]  # Flatten MultiIndex columns

# Reset the index to make 'Date' a regular column
data.reset_index(inplace=True)

# Convert 'Date' column to string format
data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')  # This will convert the date to 'YYYY-MM-DD' format

# Step 3: Create SQLite connection
conn = sqlite3.connect("stock_dataAPPL.db")  # Creates a new database file 'stock_data.db'
cursor = conn.cursor()

# Step 4: Drop the table if it exists (to avoid conflicts) and recreate it with the correct column names
cursor.execute('''
DROP TABLE IF EXISTS apple_stock;
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS apple_stock (
    Date TEXT,
    Close_AAPL REAL,
    High_AAPL REAL,
    Low_AAPL REAL,
    Open_AAPL REAL,
    Volume_AAPL INTEGER
)
''')

# Step 5: Insert data into the table
for row in data.itertuples(index=False):
    cursor.execute('''
    INSERT INTO apple_stock (Date, Close_AAPL, High_AAPL, Low_AAPL, Open_AAPL, Volume_AAPL)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (row.Date, row.Close_AAPL, row.High_AAPL, row.Low_AAPL, row.Open_AAPL, row.Volume_AAPL))

# Commit changes and close the connection
conn.commit()
conn.close()

print("Data saved to SQLite database.")


[*********************100%***********************]  1 of 1 completed

Data saved to SQLite database.


In [4]:
pip install alpha_vantage


Note: you may need to restart the kernel to use updated packages.


In [13]:
import os 

#set the environment for alpha_vantage
os.environ['Alpha_API_KEY'] = '9454UYSL8NR8WCTN'

In [14]:
import os 
import requests 

Alpha_API_Key= os.getenv("Alpha_API_KEY") 


In [6]:
import os

# Set the environment variable
os.environ['NEWS_API_KEY'] = '4967d527b00948cd8b94061cf59bc78d'

In [7]:
import os
import requests 

API_KEY = os.getenv("NEWS_API_KEY")  # Fetch from environment variable
url = f"https://newsapi.org/v2/everything?q=stock&apiKey={API_KEY}"

response = requests.get(url).json()
print(response)


{'status': 'ok', 'totalResults': 60942, 'articles': [{'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'Kyle Barr', 'title': 'Nvidia’s RTX 5060 and 5060 Ti GPUs May Arrive Real Soon', 'description': 'Take your bets whether you’ll be able to find Nvidia’s lower-end RTX 5060 and RTX 5050 in stock anywhere.', 'url': 'https://gizmodo.com/nvidias-rtx-5060-and-5060-ti-gpus-may-arrive-real-soon-2000574616', 'urlToImage': 'https://gizmodo.com/app/uploads/2025/03/Nvidia-RTX-5070-4.jpg', 'publishedAt': '2025-03-11T20:41:25Z', 'content': 'Like Bigfoot or the chupacabra, the Nvidia GeForce RTX 5060 is being spotted in the wild by ravenous GPU buyers ready to believe any hint of a footprint indicates how powerful a beast they are. If an… [+3242 chars]'}, {'source': {'id': None, 'name': 'Gizmodo.com'}, 'author': 'Gizmodo Deals', 'title': 'The Galaxy S25 Ultra Is Now 70% Off, Samsung Is Clearing Out Stock', 'description': "Samsung's latest deal on its website makes the Galaxy S25 Ultra the mu

In [15]:
import os
import requests
import sqlite3
import pandas as pd

# API Key (Ensure you've set this in your environment variables)
API_KEY = os.getenv("NEWS_API_KEY")

# Fetch news data from NewsAPI
url = f"https://newsapi.org/v2/everything?q=stock&apiKey={API_KEY}"
response = requests.get(url).json()

# Extract relevant fields
articles = response.get("articles", [])

# Convert to DataFrame
df_news = pd.DataFrame(articles)[["publishedAt", "title", "description", "url"]]

# Convert publishedAt to datetime
df_news["publishedAt"] = pd.to_datetime(df_news["publishedAt"])

# Define the relative path
#db_path = "data/stock_dataAPPL.db"

# Ensure the "data" directory exists
#os.makedirs(os.path.dirname(db_path), exist_ok=True)


# Connect to SQLite database
conn = sqlite3.connect("C:/Users/NP.MOHSENPAK/stockmarket-data-engineering/data/stock_dataAPPL.db")

# Save DataFrame to SQLite
df_news.to_sql("financial_news", conn, if_exists="replace", index=False)

print("News data saved to SQLite successfully!")

# Close connection
conn.close()


News data saved to SQLite successfully!


In [24]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\NP.MOHSENPAK\AppData\Roaming\nltk_data...


True

In [25]:
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
sentiment = sia.polarity_scores("Stock prices are rising!")
print(sentiment)


{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [2]:
!pip install sqlalchemy



In [3]:
import sqlalchemy
print(sqlalchemy.__version__)


2.0.34


In [17]:
import os
import requests
import sqlite3
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()

# API Key (Ensure you've set this in your environment variables)
API_KEY = os.getenv("NEWS_API_KEY")

# Fetch news data from NewsAPI
url = f"https://newsapi.org/v2/everything?q=stock&apiKey={API_KEY}"
response = requests.get(url).json()

# Extract relevant fields
articles = response.get("articles", [])

# Convert articles to DataFrame
df_news = pd.DataFrame(articles)[["publishedAt", "title", "description", "url"]]

# Convert publishedAt to datetime
df_news["publishedAt"] = pd.to_datetime(df_news["publishedAt"])

# Perform Sentiment Analysis on each news title and add sentiment scores
df_news["sentiment_score"] = df_news["title"].apply(lambda x: sia.polarity_scores(x)["compound"])

# Determine if sentiment is positive, neutral, or negative
df_news["sentiment_label"] = df_news["sentiment_score"].apply(lambda x: "positive" if x > 0 else ("negative" if x < 0 else "neutral"))

# Connect to SQLite database
conn = sqlite3.connect("C:/Users/NP.MOHSENPAK/stockmarket-data-engineering/data/stock_dataAPPL.db")

# Save DataFrame to SQLite (add sentiment analysis results)
df_news.to_sql("financial_news_sentiment", conn, if_exists="replace", index=False)

print("News sentiment data saved to SQLite successfully!")

# Close SQLite connection
conn.close()


News sentiment data saved to SQLite successfully!
